In [ ]:
from dotenv import load_dotenv
load_dotenv() #loads the env variables from the .env file

import os, sys
sys.path.append("..")
import emtmlibpy.emtmlibpy as emtm
from pathlib import Path

import os

In [ ]:
key1 = os.environ["EMTMLIB_KEY1"]
key2 = os.environ["EMTMLIB_KEY2"]
success = emtm.emtm_set_licence_keys(key1, key2)
assert success, "Error setting EMTMlib licence keys"

In [ ]:
print(emtm.emtm_version())

In [ ]:
def longest_common_substring(strings):
    """Find the longest common starting substring from a list of strings."""
    if not strings:
        return ''
    shortest = min(strings, key=len)
    for i in range(len(shortest)):
        for other in strings:
            if other[i] != shortest[i]:
                return shortest[:i]
    return shortest


In [ ]:
def concat_emobs(files: list[Path], output_file: str = None):
    """
    Given a list of emobs files, concatenate them and save as a new output file.
    If output_file is None, generate a filename based on common parts of input names.

    Requires write access in the licence file.
    """
    DEBUG = True
    num_files = len(files)
    out_file_id = num_files

    # Determine output path
    parent_dir = files[0].parent
    filenames = [f.stem for f in files]  # remove .emobs
    prefix = longest_common_substring(filenames).rstrip('_')
    if not output_file:
        output_file = f"{prefix}_merged.emobs"
    output_path = parent_dir / output_file

    # Create and populate the output emobs file
    emtm.em_create(em_file_id=out_file_id)

    for in_file_id, em_file in enumerate(files):
        print(f'Getting points for {em_file}')
        emtm.em_load_data(filename=str(em_file), em_file_id=in_file_id)
        _, box_count = emtm.em_point_count(em_file_id=in_file_id)

        for jj in range(box_count):
            data = emtm.em_get_point(em_file_id=in_file_id, n_index=jj)
            if DEBUG:
                print(f'file: {data.str_filename}, frame: {data.n_frame}, species: {data.str_species}')
            emtm.em_add_point(em_file_id=out_file_id, data=data)

    print(f'Writing to file: {output_path}')
    return emtm.em_write_data(em_file_id=out_file_id, filename=str(output_path))





In [ ]:
input_dir = Path('/home/marrabld/projects/emtmlibpy/src/data/dpird/250618/batch2')
emobs_files = sorted(input_dir.glob('*.emobs'))

concat_emobs(emobs_files)

